In [28]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
import datetime

import os as os
import csv

In [30]:

def existedb(url: str):
    """Funcion que verifica si una url existe en la base de datos
    Args:
        url (str): url del articulo a verificar

    Returns:
        Bool: False si la url existe, True si no existe
    """
    
    db = pd.read_csv("../data/raw/noticias.csv",encoding='utf8')
    return True if (db["URL"].eq(url)).any() else False
    # with open('../data/raw/articulos.csv', encoding="utf8") as f:
    #     csvreader = csv.reader(f, delimiter=",")
    #     for row in csvreader:
    #         if url in row[5]:
    #              return False
    #     return True


In [31]:
def obtener_autor(driver: sel.webdriver.Edge):
    """Funcion que obtiene el autor del articulo

    Args:
        driver (sel.webdriver.Edge): driver de selenium

    Returns:
        str: Nombre del autor del articulo
    """
    autor = ''
    try :
        secAutor = driver.find_element(By.XPATH,'.//div[contains(@class,"author-article")]')
    except:
        autor = 'SIN AUTOR'
    else:
        try:
            autor = secAutor.find_element(By.XPATH,'.//button').text
        except:
            autor = secAutor.find_element(By.XPATH,'.//span').text
            
    return autor
    

In [32]:
def obtener_articulos_relacionados(driver: sel.webdriver.Edge):
    """Obtener los articulos relacionados a un articulo

    Args:
        driver (sel.webdriver.Edge): referencia al driver de selenium

    Returns:
        List: lista de articulos relacionados
    """
    relNewsUrls = []
    try :
        related = driver.find_elements(By.XPATH,'.//div[contains(@class,"relatedNews")]')
        for i in related:
            relNewsUrls.append(i.find_element(By.XPATH,'.//a').get_attribute('href'))
    except:
        relNewsUrls = []
    return relNewsUrls

In [33]:
def obtener_contenido(driver: sel.webdriver.Edge):
    """Funcion que itera sobre todos los parrafos del articulo y los extrae.

    Args:
        driver (sel.webdriver.Edge): driver de selenium

    Returns:
        str: devuelve el contenido del articulo
    """
    contenido = ''
    try :
        html = driver.find_element(By.XPATH,'.//div[contains(@class,"html-content")]')
        parrafos = html.find_elements(By.XPATH,'.//p')
    except:
        contenido = 'SIN PARRAFOS'
    else:
        for i in parrafos:
            contenido += i.text
       
    return contenido

In [47]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")

In [48]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.larepublica.co/{empresa}')
time.sleep(2)

In [49]:
# Extrae la lista de todos los articulos de la pagina
articulos = driver.find_elements(By.XPATH,'.//div[contains(@class,"row news")]')

In [ ]:
#sacar 3 principales titulares
titulares = []
princip = driver.find_elements(By.XPATH,'.//div[contains(@class,"first-news")]')
urlPrinc = princip.find_elements(By.XPATH,'.//a')[1].get_attribute('href')
temaPrinc = princip.find_elements(By.XPATH,'.//a')[1].text
fechaPrinc = princip.find_element(By.XPATH,'.//span').text
tituloPrinc = princip.find_elements(By.XPATH,'.//a')[2].text
imagenPrinc = princip.find_element(By.XPATH,'.//img').get_attribute('src')
titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'Titulo':tituloPrinc,
                    'Fecha Publicacion':fechaPrinc,
                    'Tema':temaPrinc,
                    'URL':url,
                    'Imagen':imagen,
                    'Empresa':empresa})

#el resto, autor, resumen, contenido y relacionados se sacan entrando a la url
#despues de guardar articulos normales. Mismo para las otras dos noticias principales

In [37]:
# Itera por cada articulo y extrae la informacion (EN CASO DE QUE NO EXISTA ARCHIVO DE ALMACENAMIENTO ANTERIOR)
for art in articulos:
    url = art.find_elements(By.XPATH,'.//a')[1].get_attribute('href')
    fechaP = art.find_element(By.XPATH,'.//span[@class = "date-news"]').text
    tema = art.find_elements(By.XPATH,'.//a')[1].text
    resumen = art.find_element(By.XPATH,'.//p').text
    titulo = art.find_element(By.XPATH,'.//h2').text
    imagen = art.find_elements(By.XPATH,'.//img')[0].get_attribute('src')
    titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'Titulo':titulo,
                    'Fecha Publicacion':fechaP,
                    'Tema':tema,
                    'Resumen':resumen,
                    'URL':url,
                    'Imagen':imagen,
                    'Empresa':empresa})
        
    

In [50]:
# Itera por cada articulo y extrae la informacion (CASO DE QUE YA EXISTA ARCHIVO DONDE SE ALMACENA)
for art in articulos:
    url = art.find_elements(By.XPATH,'.//a')[1].get_attribute('href')
    if not(existedb(url)):
        fechaP = art.find_element(By.XPATH,'.//span[@class = "date-news"]').text
        tema = art.find_elements(By.XPATH,'.//a')[1].text
        resumen = art.find_element(By.XPATH,'.//p').text
        titulo = art.find_element(By.XPATH,'.//h2').text
        imagen = art.find_elements(By.XPATH,'.//img')[0].get_attribute('src')
        titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'Titulo':titulo,
                        'Fecha Publicacion':fechaP,
                        'Tema':tema,
                        'Resumen':resumen,
                        'URL':url,
                        'Imagen':imagen,
                        'Empresa':empresa})
    

In [51]:
# busca los autores de cada articulo y las almacena en la lista de titulares
for tit in titulares:
    
    driver.get(tit['URL'])

    # agregar autor al dict de titulares
    tit['Autor'] = obtener_autor(driver)
    
    # agregar contenido al dict de titulares
    tit['Contenido'] = obtener_contenido(driver)
    
    #agregar lista de URLs de noticias relacionadas
    tit['RelNewsUrls'] = obtener_articulos_relacionados(driver)     

    # se podria agregar un if resumen vacio, llamar a resumen. (para las 3 noticias principales)

In [52]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Tema','Resumen','URL','Imagen','Empresa','Autor','Contenido','RelNewsUrls']

In [53]:
# Para agregar titulares a un archivo.
with open('../data/raw/noticias.csv', 'a', newline='') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [41]:
#Para crear un nuevo archivo con los titulares
pd.json_normalize(titulares).to_csv(f'../data/raw/noticias.csv',index=False, encoding='utf8')